# Подготовка

In [ ]:
from abc import (
    ABC,
    abstractmethod,
)
from dataclasses import dataclass, field
from itertools import product as prod
from pprint import pprint
from time import sleep
from typing import Any, Collection, Iterator, Sequence, TypeVar
from IPython.display import display, HTML
import base64
# import numpy as np
import pandas as pd
import pygraphviz as pgv
import sympy as sm
import re

In [ ]:
State = TypeVar('State')
NewState = TypeVar('NewState')
Input = TypeVar('Input')
Output = TypeVar('Output')
Any1 = Any
Any2 = Any
StateMachine = TypeVar('StateMachine')
_dummy_graph_for_empty_node = pgv.AGraph()
_dummy_graph_for_empty_node.add_node("EMPTY", shape="none", label="")
EMPTY_NODE = pgv.Node(_dummy_graph_for_empty_node, "EMPTY")
abc = 'ABCDEFGHIJKLMNOPQRSTUVWXYZБГДЁЖЗИЙЛУФЦЧШЩЪЫЬЭЮЯ'
_CRUTCH = "-"

In [ ]:
def draw(
    *graphs: pgv.AGraph | StateMachine,
    width=200,
    labels=None,
    prog='dot'
):
    """https://stackoverflow.com/a/40494608 and DeepSeek"""

    html_content = "<div style = 'display: flex; flex-wrap: wrap; gap: 20px;'>"
    labels = labels or list(range(len(graphs)))
    for i, graph in enumerate(graphs):
        if not isinstance(graph, pgv.AGraph):
            graph = graph.get_graph()
        img_b64 = base64.b64encode(
            graph.draw(format='png', prog=prog)
        ).decode()
        img_src = f"data:image/png;base64,{img_b64}"
        html_content += f"""
<div style = 'text-align: center;'>
    <img src = '{img_src}' width = '{width}' style = 'border: 1px solid #ccc;'>
    <div><strong>{labels[i]}</strong></div>
</div>
"""
    html_content += "</div>"
    display(HTML(html_content))

In [ ]:
class Pair:
    """Пара элементов. Нужно для сравнения автоматов на эквивалентность."""

    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __lt__(self, other):
        return self.a < other.a or self.b < other.b

    def __str__(self):
        return f'({self.a}, {self.b})'

    def __repr__(self):
        return f'({self.a}, {self.b})'

    def __bool__(self):
        return bool(self.a == self.b)

    def __eq__(self, other):

        if not isinstance(other, Pair):
            return False

        return (self.a, self.b) == (other.a, other.b)

    def __hash__(self):
        return hash((self.a, self.b))


In [ ]:
def reverse_dict_of_collection(
    d: dict[Any1, Collection[Any2]]
) -> dict[Any2, Any1]:
    return {v: k for k, setOfV in d.items() for v in setOfV}


def to_str_or_pair(value):
    if type(value) is Pair:
        return value
    return str(value)

# Автоматы

In [ ]:
class MealyStateMachine:

    def __init__(self,
                 in_abc: Collection[Input],
                 out_abc: Collection[Output],
                 state_abc: Collection[State],
                 l_func: dict[tuple[State, Input], Output],
                 d_func: dict[tuple[State, Input], State],
                 start_state: State):

        self.in_abc = tuple(map(to_str_or_pair, sorted(set(in_abc))))
        self.out_abc = tuple(map(to_str_or_pair, sorted(set(out_abc))))
        self.state_abc = tuple(map(to_str_or_pair, sorted(set(state_abc))))
        self.l_func = {
            (to_str_or_pair(k1), to_str_or_pair(k2)): to_str_or_pair(v)
            for (k1, k2), v in l_func.items()
        }
        self.d_func = {
            (to_str_or_pair(k1), to_str_or_pair(k2)): to_str_or_pair(v)
            for (k1, k2), v in d_func.items()
        }
        self.current_state = self.start_state = to_str_or_pair(start_state)
        assert self.start_state in self.state_abc
        # for (s, i), o in self.l_func.items():
        #     assert s in self.state_abc
        #     assert i in self.in_abc
        #     assert o in self.out_abc
        # for (s1, i), s2 in self.d_func.items():
        #     assert s1 in self.state_abc
        #     assert s2 in self.state_abc
        #     assert i in self.in_abc
        self.table = None

    def get_table(self) -> pd.DataFrame:
        if self.table is None:
            index = pd.Index(sorted(self.state_abc), name='states')
            columns = list()
            data = dict()
            for label, letter in prod('ld', self.in_abc):
                columns.append((label, letter))
                data.setdefault((label, letter), dict())
            columns = pd.MultiIndex.from_tuples(columns)
            for letter, state in prod(self.in_abc, self.state_abc):
                if (state, letter) in self.l_func:
                    data['l', letter][state] = self.l_func[state, letter]  
                if (state, letter) in self.d_func:
                    data['d', letter][state] = self.d_func[state, letter]
            self.table = pd.DataFrame(data=data, index=index, columns=columns)
        return self.table

    def get_graph(self) -> pgv.AGraph:
        graph = pgv.AGraph(strict=False, directed=True)
        graph.add_node(EMPTY_NODE, **EMPTY_NODE.attr)
        graph.add_nodes_from(self.state_abc, color="black", shape="circle")
        graph.add_node(self.current_state, color="black", shape="circle")
        graph.add_edge(EMPTY_NODE, self.start_state)
        for state, inp in prod(self.state_abc, self.in_abc):
            if (state, inp) not in self.d_func: continue
            if (state, inp) not in self.l_func: continue
            destination = self.d_func[state, inp]
            output = self.l_func[state, inp]
            label = f"{inp}/{output}"
            graph.add_edge(state, destination, label=label)
        return graph

    def step(self, input) -> tuple[State, Input]:
        self.current_state, output = \
            self.d_func[self.current_state, input], \
            self.l_func[self.current_state, input]
        return self.current_state, output

    def restart(self):
        self.current_state = self.start_state

    def full_output(
        self, input: Collection[Input]
    ) -> tuple[Sequence[State], Sequence[Output]]:
        self.restart()
        output, state = list(), list()
        for inp in input:
            s, o = self.step(inp)
            output.append(o)
            state.append(s)
        return tuple(state), tuple(output)

    def get_minimizer(self) -> "MealyStateMachineMinimizer":
        return MealyStateMachineMinimizer(self)

    def to_moore(self) -> "MooreStateMachine":
        state_and_outputs: dict[State, list[Output]] = dict()
        # old_states_and_new_states_with_outputs
        os_a_nswo: dict[State, list[(NewState, Output)]] = dict()
        fused_l_d_func: dict[(State, Input), (State, Output, NewState)] = dict()

        new_d_func: dict[(NewState, Input), NewState] = dict()
        new_l_func: dict[NewState, Output] = dict()

        for (source_state, input), dist_state in self.d_func.items():
            state_and_outputs.setdefault(dist_state, list())
            os_a_nswo.setdefault(dist_state, list())

            output = self.l_func[source_state, input]
            if output not in state_and_outputs[dist_state]:
                state_and_outputs[dist_state].append(output)

            new_state = str(dist_state) + \
                state_and_outputs[dist_state].index(output) * '`'
            if (new_state, output) not in os_a_nswo[dist_state]:
                os_a_nswo[dist_state].append((new_state, output))

            fused_l_d_func[source_state, input] = \
                (dist_state, output, new_state)
            new_l_func[new_state] = output

        for (old_source_state, input) in fused_l_d_func:
            _, _, new_dist_state = fused_l_d_func[old_source_state, input]
            for new_source_state, _ in os_a_nswo[old_source_state]:
                new_d_func[new_source_state, input] = new_dist_state

        return MooreStateMachine(
            in_abc=self.in_abc,
            out_abc=self.out_abc,
            state_abc=set(state for state, _ in new_l_func.items()),
            d_func=new_d_func,
            l_func=new_l_func,
            start_state=self.start_state
        )

    def __or__(self, other) -> "MealyStateMachine":
        if not isinstance(other, MealyStateMachine):
            raise TypeError("other must be MealyStateMachine")
        assert self.in_abc == other.in_abc, f'{self.in_abc} = {other.in_abc}'
        assert self.in_abc == other.in_abc, f'{self.out_abc} = {other.out_abc}'
        out_abc = list(Pair(a, b) for a, b in prod(self.out_abc, other.out_abc))

        start_state = Pair(self.start_state, other.start_state)
        l_func = dict()
        d_func = dict()
        total_states = [start_state]
        for state in total_states:
            for inp in self.in_abc:
                l_func[state, inp] =\
                    Pair(self.l_func[state.a, inp],\
                         other.l_func[state.b, inp])
                new_state =\
                    Pair(self.d_func[state.a, inp],\
                         other.d_func[state.b, inp])
                d_func[state, inp] = new_state
                if new_state not in total_states:
                    total_states.append(new_state)

        return MealyStateMachine(
            self.in_abc,
            out_abc,
            total_states,
            l_func=l_func,
            d_func=d_func,
            start_state=start_state
        )


In [ ]:
class MooreStateMachine:

    def __init__(self,
                 in_abc: Collection[Input],
                 out_abc: Collection[Output],
                 state_abc: Collection[State],
                 l_func: dict[tuple[State], Output],
                 d_func: dict[tuple[State, Input], State],
                 start_state: State,
                 empty_input: Input = ''):

        self.empty_input = empty_input
        self.in_abc = \
            tuple(map(to_str_or_pair, sorted(set(in_abc))))
        self.out_abc = \
            tuple(map(to_str_or_pair, sorted(set(out_abc))))
        self.state_abc = \
            tuple(map(to_str_or_pair, sorted(set(state_abc))))
        self.l_func = {
            (to_str_or_pair(k), _CRUTCH): to_str_or_pair(v)
            for k, v in l_func.items()
        }
        self.d_func = {
            (to_str_or_pair(k1), to_str_or_pair(k2)): to_str_or_pair(v)
            for (k1, k2), v in d_func.items()
        }
        self.current_state = self.start_state = to_str_or_pair(start_state)
        assert self.start_state in self.state_abc, \
            f"{self.start_state} not in {self.state_abc}"
        for (s, _), o in self.l_func.items():
            assert s==_CRUTCH or s in self.state_abc, f'{s} not in {self.state_abc}'
            assert o in self.out_abc, f'{o} not in {self.out_abc}'
        for (s1, i), s2 in self.d_func.items():
            assert s1==_CRUTCH or s1 in self.state_abc, f'{s1} not in {self.state_abc}'
            assert s2==_CRUTCH or s2 in self.state_abc, f'{s2} not in {self.state_abc}'
            assert i in self.in_abc, f'{i} not in {self.in_abc}'
        self.table = None

    def is_descriptor(self):
        return len(self.out_abc) == 2

    def get_table(self) -> pd.DataFrame:
        if self.table is None:
            index = pd.Index(sorted(self.state_abc), name='states')
            columns = list()
            data = dict()
            data.setdefault(('l', _CRUTCH), dict())
            for letter in self.in_abc:
                columns.append(('d', letter))
                data.setdefault(('d', letter), dict())
            columns.append(('l', _CRUTCH))
            columns = pd.MultiIndex.from_tuples(columns)
            for state in self.state_abc:
                if (state, _CRUTCH) in self.l_func:
                    data['l', _CRUTCH][state] = self.l_func[state, _CRUTCH]
                for letter in self.in_abc:
                    if (state, letter) not in self.d_func: continue
                    data['d', letter][state] = self.d_func[state, letter]
            self.table = pd.DataFrame(data=data, index=index, columns=columns)
        return self.table

    def get_graph(self, chosen_out=None) -> pgv.AGraph:
        yes = False
        if chosen_out is not None:
            assert self.is_descriptor(), f'is not descriptor'
            assert chosen_out in self.out_abc, \
                f'{chosen_out} not in {self.out_abc}'
            yes = True
        graph = pgv.AGraph(strict=False, directed=True)
        graph.add_node(EMPTY_NODE, **EMPTY_NODE.attr)
        for (state, _), out in self.l_func.items():
            label = f"{state}/{out}" if not yes else state
            shape = "doublecircle" if yes and out == chosen_out else "circle"
            graph.add_node(state, label=label, color="black", shape=shape)
        graph.add_node(self.current_state, color="black")
        penwidth = 2.0 \
            if self.l_func[self.start_state, _CRUTCH] == chosen_out \
            else 1.0
        graph.add_edge(EMPTY_NODE, self.start_state, penwidth=penwidth)
        for state, inp in prod(self.state_abc, self.in_abc):
            if (state, inp) not in self.d_func: continue
            destination = self.d_func[state, inp]
            if (destination, _CRUTCH) not in self.l_func: continue
            label = f"{inp}"
            penwidth = 2.0 \
                if self.l_func[destination, _CRUTCH] == chosen_out \
                else 1.0
            graph.add_edge(state, destination, label=label, penwidth=penwidth)
        return graph

    def step(self, input) -> tuple[State, Output]:
        self.current_state = self.d_func[self.current_state, input]
        output, = self.l_func[self.current_state]
        return self.current_state, output

    def restart(self):
        self.current_state = self.start_state

    def full_output(self, input: Collection[Input]) -> tuple[tuple[State, ...], tuple[Output, ...]]:
        self.restart()
        output, state = [self.l_func[self.current_state]], [self.current_state]
        for inp in input:
            s, o = self.step(inp)
            output.append(o)
            state.append(s)
        return tuple(state), tuple(output)

    def get_minimizer(self) -> "MooreStateMachineMinimizer":
        return MooreStateMachineMinimizer(self)

    def to_mealy(self) -> MealyStateMachine:
        l_func = dict()
        for (source_state, input), dist_state in self.d_func.items():
            output = self.l_func[dist_state, _CRUTCH]
            l_func[source_state, input] = output
        return MealyStateMachine(
            self.in_abc,
            self.out_abc,
            self.state_abc,
            l_func,
            self.d_func,
            self.start_state
        )

    ### blackO
    def __or__(self, other) -> "MooreStateMachine":
        if not isinstance(other, MooreStateMachine):
            raise TypeError("other must be MooreStateMachine")
        if self.in_abc != other.in_abc:
            raise SyntaxError("lalala")
        if self.out_abc != other.out_abc:
            raise SyntaxError("lalala")
        out_abc = \
            list(Pair(a, b) for a, b in prod(self.out_abc, other.out_abc))

        start_state = Pair(self.start_state, other.start_state)
        l_func = dict()
        d_func = dict()
        total_states = [start_state]
        for state in total_states:
            for inp in self.in_abc:
                l_func[state, ] =\
                    Pair(self.l_func[state.a, ], \
                         other.l_func[state.b, ])
                new_state =\
                    Pair(self.d_func[state.a, inp], \
                         other.d_func[state.b, inp])
                d_func[state, inp] = new_state
                if new_state not in total_states:
                    total_states.append(new_state)

        return MooreStateMachine(
            self.in_abc,
            out_abc,
            total_states,
            l_func=l_func,
            d_func=d_func,
            start_state=start_state
        )

    def to_standard(
            self,
            final_out: Output,
            in_state: State = 'IN',
            out_state: State = 'OUT',
            empty_in: Input = 'ε'
        ) -> 'MooreStateMachine':
        in_abc = {empty_in, *self.in_abc}
        out_abc = {*self.out_abc}
        states = {in_state, out_state, *self.state_abc}
        non_final: Output = list(set(self.out_abc) ^ {final_out}) [0]
        l_func: dict[tuple[State], Output] = {
            st: non_final for st in self.state_abc
        }
        d_func: dict[tuple[State, Input], State] = self.d_func.copy()

        l_func[in_state] = non_final
        l_func[out_state] = final_out
        d_func[in_state, empty_in] = self.start_state

        for state in self.state_abc:
            if self.l_func[state, _CRUTCH] == final_out:
                d_func[state, empty_in] = out_state
            else:
                d_func[state, empty_in] = _CRUTCH

        return MooreStateMachine(
            in_abc,
            out_abc,
            states,
            l_func=l_func,
            d_func=d_func,
            start_state=in_state,
            empty_input = empty_in
        )
        

In [ ]:
class Undetermined:

    def __init__(self,
                 in_abc: Collection[Input],
                 state_abc: Collection[State],
                 d_func: dict[tuple[State, Input], Collection[State]],
                 start_states: Collection[State],
                 fin_states: Collection[State]):

        self.in_abc =\
            tuple(map(to_str_or_pair, sorted(set(in_abc))))
        self.state_abc =\
            tuple(map(to_str_or_pair, sorted(set(state_abc))))
        self.d_func = {
            (to_str_or_pair(k1), to_str_or_pair(k2)):
            frozenset(map(to_str_or_pair, sorted(set(v))))
            for (k1, k2), v in d_func.items()
        }
        self.start_states =\
            tuple(map(to_str_or_pair, sorted(set(start_states))))
        self.fin_states =\
            tuple(map(to_str_or_pair, sorted(set(fin_states))))
        for state in self.start_states:
            assert state in self.state_abc, f'{state} not in {self.state_abc}'
        for (s1, i), s2 in self.d_func.items():
            assert s1 in self.state_abc, f'{s1} not in {self.state_abc}'
            assert i in self.in_abc, f'{i} not in {self.in_abc}'
            for s in s2:
                assert s in self.state_abc, f'{s} not in {self.state_abc}'

        for s in self.start_states:
            assert s in self.state_abc, f'{s} not in {self.state_abc}'
        for s in self.fin_states:
            assert s in self.state_abc, f'{s} not in {self.state_abc}'
        self.table = None

    def get_g_and_m(self) -> tuple[dict, list]:
        m: list[frozenset[State]] = list()
        g: dict[tuple[frozenset[State], Input], frozenset[State]] = dict()

        m.append(frozenset(self.start_states))
        for m_states in m:
            for input in self.in_abc:
                x_states = frozenset()
                for m_state in m_states:
                    x_states |= self.d_func.get((m_state, input), set())
                g[m_states, input] = x_states
                if x_states not in m:
                    m.append(x_states)

        return g, m

    def get_table(self) -> pd.DataFrame:
        if self.table is None:
            g, m = self.get_g_and_m()

            # new_states = {state: abc[i] for i, state in enumerate(m)}
            # new_start_state = new_states[frozenset(self.start_states)]

            index = pd.Index(m, name='-')
            columns = list()
            data = dict()
            for letter in self.in_abc:
                columns.append(letter)
                data.setdefault(letter, list())
            for m_states, input in prod(m, self.in_abc):
                data[input].append(g[m_states, input])
            columns = pd.Index(columns)
            self.table = pd.DataFrame(
                data=data, index=index, columns=columns, dtype='object'
            )
        return self.table

    def get_graph(self) -> pgv.AGraph:
        graph = pgv.AGraph(strict=False, directed=True)
        for state in self.state_abc:
            label = state
            shape = "doublecircle" if state in self.fin_states else "circle"
            graph.add_node(state, label=label, color="black", shape=shape)
            if state in self.start_states:
                graph.add_node(f'empty_{state}', **EMPTY_NODE.attr)
                penwidth = 2.0 if state in self.fin_states else 1.0
                graph.add_edge(f'empty_{state}', state, penwidth=penwidth)
        for state, inp in prod(self.state_abc, self.in_abc):
            if (state, inp) not in self.d_func.keys():
                continue
            for destination in self.d_func[state, inp]:
                label = f"{inp}"
                penwidth = 2.0 if destination in self.fin_states else 1.0
                graph.add_edge(state, destination, label=label, penwidth=penwidth)
        return graph

    def to_moore(self) -> MooreStateMachine:
        g, m = self.get_g_and_m()

        new_states = {state: abc[i] for i, state in enumerate(m)}
        new_start_state = new_states[frozenset(self.start_states)]

        return MooreStateMachine(
            in_abc=self.in_abc,
            out_abc=(0, 1),
            state_abc=[v for _, v in new_states.items()],
            l_func={
                new_states[states]:
                int(bool(states & frozenset(self.fin_states)))
                for states in m
            },
            d_func={
                (new_states[s], i): new_states[d] for (s, i), d in g.items()
            },
            start_state=new_start_state
        )

# Минимизаторы автоматов

In [ ]:
class MealyStateMachineMinimizer:

    def __init__(self, machine: MealyStateMachine):
        self.machine = machine
        self.klasses = [{'A0': self.machine.state_abc}]

        self.d_funcs: dict[tuple['d{n}', State, Input], 'A0'] = dict()
        self.iter = 0

        self.table = None
        self.minimized_machine = None

    def get_table(self) -> pd.DataFrame:
        if self.table is None:
            index = pd.Index(sorted(self.machine.state_abc), name='states')
            columns = list()
            data = dict()
            for label, letter in prod(range(1, len(self.klasses)), self.machine.in_abc):
                columns.append((f'd{label}', letter))
                data.setdefault((f'd{label}', letter), dict())
            columns = pd.MultiIndex.from_tuples(columns)
            for (d, state, inp), new_state in self.d_funcs.items():
                data[d, inp][state] = new_state
            self.table = pd.DataFrame(data=data, index=index, columns=columns)
        return self.table

    def get_minimal(self) -> MealyStateMachine:
        if self.minimized_machine is None:
            while not self.is_minimal():
                self.iter += 1
                self.get_n_klasses(self.iter)
            klasses = self.get_n_klasses(-1)
            Smin = tuple(set(klasses.keys()))
            rev = reverse_dict_of_collection(klasses)
            l_func_min = dict()
            d_func_min = dict()
            start_state_min = None
            for state in Smin:
                if self.machine.start_state in klasses[state]:
                    start_state_min = state
                original_state = list(klasses[state])[0]
                for inp in self.machine.in_abc:
                    if inp in self.machine.get_table()['l'].keys():
                        l_func_min[state, inp] =\
                            self.machine.get_table()['l', inp][original_state]
                    if inp in self.machine.get_table()['d'].keys() \
                        and original_state in self.machine.get_table()['d', inp] \
                        and self.machine.get_table()['d', inp][original_state] in rev:
                        d_func_min[state, inp] =\
                            rev[self.machine.get_table()['d', inp][original_state]]
            self.minimized_machine = MealyStateMachine(
                in_abc=self.machine.in_abc,
                out_abc=self.machine.out_abc,
                state_abc=Smin,
                d_func=d_func_min,
                l_func=l_func_min,
                start_state=start_state_min
            )
        return self.minimized_machine

    def is_minimal(self) -> bool:
        if self.minimized_machine is not None:
            return True
        for n, i in enumerate(self.klasses[:-1]):
            if set(i.values()) == set(self.klasses[-1].values()):
                return True
        return False

    def get_all_klasses(self) -> list[dict[str, Collection[State]]]:
        return self.klasses

    def get_n_klasses(self, n: int = -1) -> dict[str, Collection[State]]:
        if n >= len(self.klasses):
            grouped_states: \
                dict[tuple[Output], set[State]] | \
                dict[tuple[Any[Output, State]], set[State]] | \
                dict[tuple[Any[Output, State]], tuple[State]] \
                = dict()

            for state in self.klasses[0]['A0']:
                # Какие выходы у этого состояния
                outputs_and_states = tuple(
                    self.machine.l_func[state, inp]
                    for inp in self.machine.in_abc
                    if (state, inp) in self.machine.l_func
                )
                if n > 1:
                    # Какие выходы И d{n-1} состояния у этого состояния
                    outputs_and_states += tuple(
                        self.d_funcs[f'd{n-1}', state, inp]
                        for inp in self.machine.in_abc
                        if (f'd{n-1}', state, inp) in self.d_funcs
                    )
                grouped_states.setdefault(outputs_and_states, set())
                grouped_states[outputs_and_states].add(state)
            for k, v in grouped_states.items():
                grouped_states[k] = tuple(sorted(v))
            klasses: dict['A0', tuple[State]] = dict()

            for k, v in grouped_states.items():
                klasses[abc[len(klasses)]+str(n)] = v

            self.klasses.append(klasses)
            rev: dict[State, 'A0'] = reverse_dict_of_collection(klasses)
            for state, inp in prod(self.machine.state_abc, self.machine.in_abc):
                if (state, inp) not in self.machine.d_func: continue
                if (f'd{n}', state, inp) not in self.d_funcs: continue
                next_state = self.machine.d_func[state, inp]
                self.d_funcs[f'd{n}', state, inp] = rev[next_state]

        elif n < 0:
            return {k[0]: v for k, v in self.klasses[-1].items()}

        return self.klasses[n]

In [ ]:
class MooreStateMachineMinimizer:

    def __init__(self, machine: MooreStateMachine):
        self.machine = machine
        self.klasses = [{'A0': self.machine.state_abc}]

        self.d_funcs: dict['d{n}', State, Input, 'A0'] = dict()
        self.iter = 0

        self.table = None
        self.minimized_machine = None

    def get_table(self) -> pd.DataFrame:
        if self.table is None:
            index = pd.Index(sorted(self.machine.state_abc), name='states')
            columns = list()
            data = dict()
            for label, letter in prod(
                range(1, len(self.klasses)), self.machine.in_abc
            ):
                columns.append((f'd{label}', letter))
                data.setdefault((f'd{label}', letter), dict())
            columns = pd.MultiIndex.from_tuples(columns)
            for (d, state, inp), new_state in self.d_funcs.items():
                data[d, inp][state] = new_state
            self.table = pd.DataFrame(data=data, index=index, columns=columns)
        return self.table

    def get_minimal(self) -> MooreStateMachine:
        if self.minimized_machine is None:
            while not self.is_minimal():
                self.iter += 1
                self.get_n_klasses(self.iter)
            klasses = self.get_n_klasses(-1)
            Smin = tuple(set(klasses.keys()))
            rev = reverse_dict_of_collection(klasses)
            l_func_min = dict()
            d_func_min = dict()
            start_state_min = None

            for state in Smin:
                if self.machine.start_state in klasses[state]:
                    start_state_min = state
                original_state = list(klasses[state])[0]
                l_func_min[state] =\
                    self.machine.get_table()['l', _CRUTCH][original_state]
                for inp in self.machine.in_abc:
                    d_func_min[state, inp] =\
                        rev[self.machine.get_table()['d', inp][original_state]]
            self.minimized_machine = MooreStateMachine(
                in_abc=self.machine.in_abc,
                out_abc=self.machine.out_abc,
                state_abc=Smin,
                d_func=d_func_min,
                l_func=l_func_min,
                start_state=start_state_min
            )
        return self.minimized_machine

    def is_minimal(self) -> bool:
        if self.minimized_machine is not None:
            return True
        for n, i in enumerate(self.klasses[:-1]):
            if set(i.values()) == set(self.klasses[-1].values()):
                return True
        return False

    def get_all_klasses(self) -> list[dict[str, Collection[State]]]:
        return self.klasses

    def get_n_klasses(self, n: int = -1) -> dict[str, Collection[State]]:
        if n >= len(self.klasses):
            grouped_states: \
                dict[tuple[Output], set[State]] | \
                dict[tuple[Any[Output, State]], set[State]] | \
                dict[tuple[Any[Output, State]], tuple[State]] \
                = dict()

            for state in self.klasses[0]['A0']:

                print(self.d_funcs)
                # Какие выходы у этого состояния
                outputs_and_states = tuple(self.machine.l_func[state, _CRUTCH])
                if n > 1:
                    # Какие выходы И d{n-1} состояния у этого состояния
                    outputs_and_states += tuple(
                        self.d_funcs[f'd{n-1}', state, inp]
                        for inp in self.machine.in_abc
                    )

                grouped_states.setdefault(outputs_and_states, set())
                grouped_states[outputs_and_states].add(state)

            for k, v in grouped_states.items():
                grouped_states[k] = tuple(sorted(v))

            klasses: dict['A0', tuple[State]] = dict()

            for k, v in grouped_states.items():
                klasses[abc[len(klasses)]+str(n)] = v

            self.klasses.append(klasses)
            rev: dict[State, 'A0'] = reverse_dict_of_collection(klasses)
            for state, inp in prod(self.machine.state_abc, self.machine.in_abc):
                if (state, inp) not in self.machine.d_func: continue
                if (f'd{n}', state, inp) not in self.d_funcs: continue
                next_state = self.machine.d_func[state, inp]
                self.d_funcs[f'd{n}', state, inp] = rev[next_state]

        elif n < 0:
            return {k[0]: v for k, v in self.klasses[-1].items()}

        return self.klasses[n]

# Для задания 4

In [ ]:
@dataclass(frozen=True)
class Expression(ABC):

    @property
    @abstractmethod
    def name(self) -> str:
        ...

    @classmethod
    @abstractmethod
    def is_final(cls) -> bool:
        ...

    @abstractmethod
    def __iter__(self) -> Iterator['Expression']:
        ...

    @abstractmethod
    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        ...

    @abstractmethod
    def __str__(self) -> str:
        ...

    @abstractmethod
    def __repr__(self) -> str:
        ...


@dataclass(frozen=True)
class Operator(Expression, ABC):

    @property
    @abstractmethod
    def args_number(self) -> int:
        ...

    @property
    @abstractmethod
    def args(self) -> tuple[Expression, ...]:
        ...

    @classmethod
    def is_final(cls) -> bool:
        return False

    def __repr__(self):
        return f'{self.name}({", ".join(repr(arg) for arg in self.args)})'

    def __iter__(self):
        yield self
        for arg in self.args:
            for i in arg:
                yield i

    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        if self == old:
            return new
        args = tuple(arg.replace(old, new) for arg in self.args)
        if self.args_number == 1:
            args = args[0]
        return type(self)(args)


@dataclass(frozen=True)
class Function(Operator, ABC):

    def __str__(self):
        return f'{self.name}({", ".join(
            str(arg)
            if arg.is_final() or self.args_number == 1
            else f"({arg})"
            for arg in self.args
        )})'


@dataclass(frozen=True)
class InfixOperator(Operator, ABC):

    def __str__(self):
        return f"{self.name}".join(
            str(arg)
            if arg.is_final()
            or isinstance(arg, Function)
            or isinstance(arg, Multiplication)
            or isinstance(arg, Repetition)
            else f"({arg})"
            for arg in self.args
        )


@dataclass(frozen=True)
class PostfixOperator(Operator, ABC):

    def __str__(self):
        res = f", ".join(
            str(arg)
            if arg.is_final()
            or isinstance(arg, Function)
            else f"({arg})"
            for arg in self.args
        )
        if len(self.args) > 1:
            res = f'({res})'
        return  res  + self.name


@dataclass(frozen=True)
class Literal(Expression):
    _name: str = field()

    @property
    def name(self) -> str:
        return self._name

    @classmethod
    def is_final(cls) -> bool:
        return True

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'Literal({repr(self.name)})'

    def __iter__(self) -> 'Literal':
        yield self

    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        return new if self == old else old

@dataclass(frozen=True)
class Addition(InfixOperator):
    _args: tuple[Expression, ...]

    @property
    def args_number(self) -> int:
        return len(self.args)

    @property
    def args(self) -> tuple[Expression, ...]:
        return self._args

    @property
    def name(self) -> str:
        return '+'
    
    def replace(self, old: Expression, new: Expression) -> Expression:
        return self


@dataclass(frozen=True)
class Multiplication(InfixOperator):
    _args: tuple[Expression, ...]

    @property
    def args_number(self) -> int:
        return len(self.args)

    @property
    def args(self) -> tuple[Expression, ...]:
        return self._args

    @property
    def name(self) -> str:
        return ''


@dataclass(frozen=True)
class Repetition(PostfixOperator):
    _arg: Expression

    @property
    def args_number(self) -> int:
        return 1

    @property
    def args(self) -> tuple[Expression]:
        return self._arg,

    @property
    def name(self) -> str:
        return '*'

# Какой-то хлам

In [ ]:
temp = MooreStateMachine('ab', 'dc', range(1,5), l_func = {
    1:'c',2:'d',3:'d',4:'c'
}, d_func = {
    (1, 'a'):2,
    (1, 'b'):3,
    (2, 'a'):2,
    (2, 'b'):4,
    (3, 'a'):1,
    (3, 'b'):2,
    (4, 'a'):1,
    (4, 'b'):2,
},start_state = 1)
temp2 = temp.to_mealy()
print(temp.get_table())
print(temp2.get_table())
draw(temp, temp2,width = 400, prog = 'circo')

In [ ]:
def get_alpha():

    # States
    STATES = ['0', '1', '2', '3', '4']

    # Input alphabet
    INPUTS = ['a', 'b', 'c']

    # Output alphabet
    OUTPUTS = ['0', '1']

    # Output function: {(INPUT, STATE): OUTPUT}
    output_function = {
        ('0', 'a'): '0',
        ('0', 'b'): '1',
        ('0', 'c'): '0',
        ('1', 'a'): '1',
        ('1', 'b'): '0',
        ('1', 'c'): '1',
        ('2', 'a'): '0',
        ('2', 'b'): '0',
        ('2', 'c'): '1',
        ('3', 'a'): '1',
        ('3', 'b'): '0',
        ('3', 'c'): '1',
        ('4', 'a'): '1',
        ('4', 'b'): '1',
        ('4', 'c'): '0'
}

    # State change function: {(INPUT, STATE): NEXT_STATE}
    state_change_function = {
        ('0', 'a'): '1',
        ('0', 'b'): '2',
        ('0', 'c'): '0',
        ('1', 'a'): '2',
        ('1', 'b'): '3',
        ('1', 'c'): '2',
        ('2', 'a'): '3',
        ('2', 'b'): '0',
        ('2', 'c'): '4',
        ('3', 'a'): '4',
        ('3', 'b'): '0',
        ('3', 'c'): '3',
        ('4', 'a'): '0',
        ('4', 'b'): '1',
        ('4', 'c'): '2'}

    return MealyStateMachine(
        in_abc = ''.join(INPUTS),
        out_abc = ''.join(OUTPUTS),
        state_abc = STATES,
        l_func = output_function,
        d_func = state_change_function,
        start_state = STATES[0]
    )

def get_beta():
    # State Machine Definition

    # States
    STATES = ['A', 'B', 'C', 'D']

    # Input alphabet
    INPUTS = ['a', 'b', 'c']

    # Output alphabet
    OUTPUTS = ['0', '1']

    # Output function: {(INPUT, STATE): OUTPUT}
    output_function = {
        ('A', 'a'): '0',
        ('A', 'b'): '1',
        ('A', 'c'): '0',
        ('B', 'a'): '1',
        ('B', 'b'): '0',
        ('B', 'c'): '1',
        ('C', 'a'): '1',
        ('C', 'b'): '0',
        ('C', 'c'): '1',
        ('D', 'a'): '1',
        ('D', 'b'): '0',
        ('D', 'c'): '1'
}

    # State change function: {(INPUT, STATE): NEXT_STATE}
    state_change_function = {
        ('A', 'a'): 'A',
        ('A', 'b'): 'C',
        ('A', 'c'): 'B',
        ('B', 'a'): 'B',
        ('B', 'b'): 'A',
        ('B', 'c'): 'C',
        ('C', 'a'): 'A',
        ('C', 'b'): 'C',
        ('C', 'c'): 'D',
        ('D', 'a'): 'A',
        ('D', 'b'): 'C',
        ('D', 'c'): 'B'
}
    
    return MealyStateMachine(
        in_abc = ''.join(INPUTS),
        out_abc = ''.join(OUTPUTS),
        state_abc = STATES,
        l_func = output_function,
        d_func = state_change_function,
        start_state = STATES[0]
    )

In [ ]:
def minimize_machine(machine: StateMachine):
    print("Минимизация конечного автомата")
    print(f'A = {(machine.in_abc)}')
    print(f'B = {(machine.out_abc)}')
    print(f'S = {(machine.state_abc)}')
    print(f'u0 = {machine.start_state}')
    minimalizer = machine.get_minimizer()
    minimal = minimalizer.get_minimal()
    print(machine.get_table().assign(**{'|':'|'}).join(minimalizer.get_table()))
    print("Разбиения (семейства классов):")
    n = len(minimalizer.get_all_klasses()) - 1
    j = None
    for i, k in enumerate(minimalizer.get_all_klasses()):
        if j is None and set(k.values()) == set(minimalizer.get_n_klasses(n).values()):
            j = i
        print(f'K{i} = {k}')
    print(f"Семейства классов K{n} неотличимы от K{j} => получено предельное разбиение.")
    print("Никакие слова произвольных длин не различат состояния в предельных классах.")
    print(f"Назовем их K = {minimalizer.get_n_klasses()}.")
    print('Интерпретируем эти классы как состояния минимального автомата.')
    print('Для получения функций минимального автомата берем любой представитель класса.')
    print('Упорядочим строки по списку классов и получаем минимальный автомат:')
    print(f'A = {(minimal.in_abc)}')
    print(f'B = {(minimal.out_abc)}')
    print(f'S = {(minimal.state_abc)}')
    print(f'u0 = {minimal.start_state}')
    print(minimal.get_table())
    draw(machine,minimal, width = 380, labels = ["Начальный автомат", "Минимальный автомат"])
    return minimalizer

In [ ]:
def are_machines_equal(machine1: StateMachine, machine2: StateMachine):
    fused = machine1 | machine2
    print("Автомат alpha:")
    print(machine1.get_table())
    print()
    print("Автомат beta:")
    print(machine2.get_table())
    print()
    print("Автомат alpha * beta:")
    print(fused.get_table())
    print()
    if fused.get_table()['l'].all(None):
        print("Значения у всех пар выходной функции одинаковы, поэтому автоматы alpha и beta эквивалентны.")
    else:
        print("Значения пар выходной функции не везде одинаковы, поэтому автоматы alpha и beta не эквивалентны.")
    draw(machine1, machine2, width = "300", labels = ["alpha", 'beta'])
    draw(fused, width = "650", labels = ['alpha * beta'])
    return fused

# Задания

## Задание 1

In [ ]:
def task1():
    alpha = get_alpha()
    beta = get_beta()
    are_machines_equal(alpha, beta)
task1()

In [ ]:
are_machines_equal(MealyStateMachine('ab',range(2),range(1,4),l_func = {
    (1, 'a'):1,
    (1, 'b'):0,
    (2, 'a'):0,
    (2, 'b'):1,
    (3, 'a'):0,
    (3, 'b'):0,
},d_func = {
    (1, 'a'):2,
    (1, 'b'):3,
    (2, 'a'):3,
    (2, 'b'):2,
    (3, 'a'):1,
    (3, 'b'):2,
},start_state = 1),MealyStateMachine('ab',range(2),'ABCDE',l_func = {
    ('A', 'a'):1,
    ('A', 'b'):0,
    ('B', 'a'):0,
    ('B', 'b'):1,
    ('C', 'a'):0,
    ('C', 'b'):0,
    ('D', 'a'):0,
    ('D', 'b'):1,
    ('E', 'a'):0,
    ('E', 'b'):0,
},d_func = {
    ('A', 'a'):'B',
    ('A', 'b'):'E',
    ('B', 'a'):'C',
    ('B', 'b'):'B',
    ('C', 'a'):'A',
    ('C', 'b'):'D',
    ('D', 'a'):'C',
    ('D', 'b'):'B',
    ('E', 'a'):'A',
    ('E', 'b'):'D',
},start_state = 'A'))


## 2

In [ ]:
def task2():
    A = 'ab'
    B = {0,1}
    S = {1,2,3,4,5,6,7,8}
    l = {
        (1, 'a'): 1,
        (2, 'a'): 0,
        (3, 'a'): 1,
        (4, 'a'): 0,
        (5, 'a'): 0,
        (6, 'a'): 1,
        (7, 'a'): 1,
        (8, 'a'): 1,
        (1, 'b'): 0,
        (2, 'b'): 1,
        (3, 'b'): 0,
        (4, 'b'): 1,
        (5, 'b'): 1,
        (6, 'b'): 0,
        (7, 'b'): 0,
        (8, 'b'): 0,
    }

    d = {
        (1, 'a'): 2,
        (2, 'a'): 3,
        (3, 'a'): 2,
        (4, 'a'): 1,
        (5, 'a'): 1,
        (6, 'a'): 5,
        (7, 'a'): 8,
        (8, 'a'): 2,
        (1, 'b'): 6,
        (2, 'b'): 5,
        (3, 'b'): 4,
        (4, 'b'): 5,
        (5, 'b'): 5,
        (6, 'b'): 7,
        (7, 'b'): 4,
        (8, 'b'): 6,
    }
    u0 = 1
    automat = MealyStateMachine(A,B,S,l,d,u0)
    minimalizer = minimize_machine(automat)
    return automat, minimalizer


a1, a2 = task2()


In [ ]:
temp1 = a2.minimized_machine.to_moore()
# print(a2.minimized_machine.get_graph())
# print(temp1.get_graph())
# pprint(temp1.l_func)
# pprint(temp1.d_func)
# draw(a2.minimized_machine,width=300)
# draw(temp1, width=300)
draw(a1, a2.minimized_machine, temp1, width=250)


In [ ]:
def test_to_moore():
    mealy = MealyStateMachine(
    in_abc='ab', out_abc='cd',state_abc=(1,2,3),start_state=1,
    l_func={
        (1,'a'):'d',
        (1,'b'):'d',
        (2,'a'):'d',
        (2,'b'):'c',
        (3,'a'):'c',
        (3,'b'):'d',
    },d_func={
        (1,'a'):2,
        (1,'b'):3,
        (2,'a'):2,
        (2,'b'):3,
        (3,'a'):1,
        (3,'b'):2,
    })
    
    moore = mealy.to_moore()
    new_mealy = moore.to_mealy()
    min_mealy = new_mealy.get_minimizer().get_minimal()
    print(min_mealy.get_table())
    print(min_mealy.start_state)
    print(new_mealy.start_state)
    draw(mealy, moore.get_graph('c'), moore.get_graph('d'), new_mealy, min_mealy,width=180)
test_to_moore()

## 3

In [ ]:
def task3():
    temp = Undetermined(
    in_abc='ab',
    start_states={2, 4},
    state_abc={1, 2, 3, 4, 5, 6},
    fin_states={3, 4},
    d_func={
        (1,'a'): {1, 2},
        (1,'b'): {3},
        
        (2,'a'): {1},
        (2,'b'): {2, 4},
        
        (3,'a'): {4},
        
        (4,'a'): {5},
        
        (5,'b'): {4},
        
        (6,'a'): {1, 3},
        (6,'b'): {2, 5},
    }
)
    print('Построение детерминированного автомат из недетерминированного.')
    print(f'Начальные вершины: {tuple(temp.start_states)}')
    g, m = temp.get_g_and_m()
    pprint(temp.get_table())
    print('Переименуем множества в состояния.')
    pprint({abc[i]: tuple(states) for i, states in enumerate(m)})
    print(f'и укажем функцию выхода: λ=1 у тех множеств, которые содержат финальные вершины исходной диаграммы ({tuple(temp.fin_states)})')
    print()
    print('Построенный детерминированный автомат:')
    moore = temp.to_moore()
    print(f'u0 = {moore.start_state}')
    pprint(moore.get_table())
    draw(temp, moore.get_graph('1'), width=400, prog='circo')
task3()


## 4

In [ ]:
def task4():
    moore = MooreStateMachine(
        in_abc='ab',
        out_abc='01',
        state_abc=('1', '2', '3', '4'),
        start_state=1,
        l_func={
            '1':'1',
            '2':'0', 
            '3':'1',
            '4':'0',
        },d_func={
            ('1', 'a'): '2',
            ('1', 'b'): '3',
            ('2', 'a'): '3',
            ('2', 'b'): '2',
            ('3', 'a'): '2',
            ('3', 'b'): '4',
            ('4', 'a'): '1',
            ('4', 'b'): '1',
        }
    )
    # print(moore.get_table())
    stan = moore.to_standard(
        final_out='1',
        in_state='0',
        out_state='5',
        empty_in='ε'
    )
    # print(str(stan.get_table()).replace('NaN', '   '))
    # draw(moore.get_graph('1'), stan.get_graph('1'), width=700, prog='circo')
    return moore, stan
moore, stan = task4()
mealy = moore.to_mealy() 
minim = mealy.get_minimizer().get_minimal()
print(moore.get_table())
print(mealy.get_table())
print(minim.get_table())

In [ ]:
# def test_to_standard():
#     moore = MooreStateMachine(
#         'ab',
#         '01',
#         '1234',
#         {'1':'1','2':'0','3':'0','4':'1'},
#         {
#             ('1','a'): '2',
#             ('1','b'): '3',
#             ('2','a'): '4',
#             ('2','b'): '4',
#             ('3','a'): '3',
#             ('3','b'): '2',
#             ('4','a'): '3',
#             ('4','b'): '4',
#         },
#         '1'
#     )
#     print(moore.get_table())
#     stan = moore.to_standard(
#         final_out='1',
#         in_state='0',
#         out_state='5',
#         empty_in='ε'
#     )
#     print(stan.get_table())
#     draw(moore.get_graph('1'), stan.get_graph('1'), width=700, prog='circo')

#     return moore, stan
# moore, stan = test_to_standard()

In [ ]:
empty = 'ε'
def get_edge_labels(graph: pgv.AGraph) -> dict[tuple[pgv.Edge, pgv.Edge], set[Expression]]:
    labels: dict[tuple[pgv.Edge, pgv.Edge], set[Expression]]= dict()
    for edge in graph.edges_iter():
        s, d = tuple([*edge])
        label = edge.attr['label']
        try:
            label = eval(label)
        except:
            pass
        if type(label) is str:
            label = Literal(label)
        labels.setdefault((s, d), set())
        labels[s,d].add(label)
    return labels

def group_double_edges(graph: pgv.AGraph, empty: Literal | str | None = None) -> tuple[pgv.AGraph, pgv.AGraph]:
    new_display = graph.copy()
    new_expression = graph.copy()

    if type(empty) is str:
        empty = Literal(empty)

    n = get_edge_labels(graph)
    for (s, d), labels in n.items():
        for label in labels:
            new_display.remove_edge(s, d)
            new_expression.remove_edge(s, d)

        if empty in labels:
            labels.remove(empty)

        if len(labels) == 0:
            label = Literal('')
        elif len(labels) > 1:
            label = Addition(tuple(labels))
        else:
            label = labels.pop()

        new_display.add_edge(s, d, label=str(label))
        new_expression.add_edge(s, d, label=repr(label))

    return new_display, new_expression

def remove_node(graph: pgv.AGraph, node, empty: Literal | str | None = None) -> tuple[pgv.AGraph, pgv.AGraph]:
    new_display = graph.copy()
    new_expression = graph.copy()
    node = graph.get_node(node)
    new_display.remove_node(node)
    new_expression.remove_node(node)

    if type(empty) is str:
        empty = Literal(empty)

    prefixes: dict[pgv.Edge, Expression] = dict()
    postfixes: dict[pgv.Edge, Expression] = dict()
    loop: Repetition | None = None
    
    n = get_edge_labels(graph)
    for (s, d), labels in n.items():
        assert len(labels) == 1
        for label in labels:
            
            try:
                label = eval(label)
            except: ...
            if type(label) is not Literal:
                label.replace(empty, Literal(''))
            # elif label == empty:
            #     label = Literal('')
            if node in (s, d):
                if d == s:
                    loop = Repetition(label)
                elif d == node:
                    prefixes[s] = label
                elif s == node:
                    postfixes[d] = label
            else:
                new_display.remove_edge(s, d)
                new_expression.remove_edge(s, d)
                new_display.add_edge(s, d, label=str(label))
                new_expression.add_edge(s, d, label=repr(label))

    for s, d in prod(prefixes.keys(), postfixes.keys()):
        labels = list()
        labels.append(prefixes[s])
        if loop:
            labels.append(loop)
        labels.append(postfixes[d])
        label = tuple(i for i in labels if i != empty)
        if len(label) == 0:
            label = empty
        elif len(label) == 1:
            label = label[0]
        else:
            label = Multiplication(label)
        label_display = str(label)
        label_expression = repr(label)
        new_display.add_edge(s, d, label=label_display)
        new_expression.add_edge(s, d, label=label_expression)

    return new_display, new_expression

WIDTH = 500
disp = stan.get_graph('1')
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = group_double_edges(disp)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = remove_node(expr, 4, empty)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = remove_node(expr, 2, empty)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = group_double_edges(expr)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = remove_node(expr, 1, empty)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = group_double_edges(expr)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = remove_node(expr, 3, empty)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
disp, expr = group_double_edges(expr)
draw(disp, prog='circo', width=WIDTH, labels=('disp',))
reg = eval(expr.edges()[1].attr['label'])
print(reg)

In [ ]:
print(reg)

### ---

In [ ]:
graph = pgv.AGraph(
    strict=False,
    directed=True,
    compound=True,
    rankdir='LR',
    ranksep=1
)
with graph.add_subgraph(name='A', cluster=True) as G:
    G.add_node(0, label='', shape='none')
    G.add_node(1, label='', color='black', shape='circle')
    G.add_node(2, label='', color='black', shape='circle')
    G.add_node(3, label='', color='black', shape='circle')
    G.add_node(4, label='', color='black', shape='circle')
    G.add_node(5, label='', color='black', shape='circle', penwidth=3)
    G.add_edge(0, 1, color='black')
    G.add_edge(1, 2, color='black', label='a')
    G.add_edge(2, 3, color='black')
    G.add_edge(3, 3, color='black', label='b')
    G.add_edge(3, 4, color='black')
    G.add_edge(4, 5, color='black', label='a')
    G.add_edge(1, 5, color='black', label='b')
    # G.draw("test.png","png", prog="dot")
    draw(G, width=400)
# print(graph)
# draw(graph, prog='dot', width=600)
# print(graph)

# А дальше я забил и сделал граф вручную

In [ ]:
graph = pgv.AGraph(
    strict=False,
    directed=True,
    compound=True,
    rankdir='LR',
    ranksep=1
)
with graph.add_subgraph(name='A', cluster=True) as G:
    G.add_node(0, label='', shape='none')
    G.add_node(1, label='', color='black', shape='circle')
    G.add_node(2, label='', color='black', shape='circle')
    # G.add_node(3, label='', color='black', shape='circle')
    # G.add_node(4, label='', color='black', shape='circle')
    G.add_node(5, label='', color='black', shape='circle', penwidth=3)
    G.add_edge(0, 1, color='black')
    G.add_edge(1, 2, color='black', label='b')
    G.add_edge(2, 5, color='black', label='a')
    G.add_edge(2, 5, color='black', label='b')
    G.add_edge(1, 5, color='black', label='ε')
    # G.add_edge(2, 3, color='black')
    # G.add_edge(3, 3, color='black', label='b')
    # G.add_edge(3, 4, color='black')
    # G.add_edge(4, 5, color='black', label='a')
    # G.add_edge(1, 5, color='black', label='b')
    G.draw("test.png","png", prog="dot")
    draw(G, width=600)

In [ ]:
graph = pgv.AGraph(
    strict=False,
    directed=True,
    compound=True,
    rankdir='LR',
    ranksep=1
)
with graph.add_subgraph(name='A', cluster=True) as G:
    G.add_node('s', label='', shape='none')
    G.add_node('e', label='', color='black', shape='circle', penwidth=3)
    for i in range(10):
        G.add_node(i, label='', color='black', shape='circle')
    # G.add_edge('s', 0, color='black')
    G.add_edge('s', 1, color='black')
    G.add_edge(1, 2, color='black', label='a')
    G.add_edge(2, 3, color='black')
    G.add_edge(3, 3, color='black', label='b')
    G.add_edge(3, 4, color='black')
    G.add_edge(4, 'e', color='black', label='a')
    G.add_edge(1, 5, color='black', label='b')
    G.add_edge(5, 6, color='black', label='a')
    G.add_edge(5, 6, color='black', label='b')
    G.add_edge(6, 7, color='black', label='a')
    G.add_edge(6, 'e', color='black', label='b')
    G.add_edge(7, 8, color='black')
    G.add_edge(8, 8, color='black', label='b')
    G.add_edge(8, 9, color='black')
    G.add_edge(9, 'e', color='black', label='a')

    # G.draw("test.png","png", prog="dot")
    draw(G, width=600)

In [ ]:
graph = pgv.AGraph(
    strict=False,
    directed=True,
    compound=True,
    rankdir='LR',
    ranksep=1
)
with graph.add_subgraph(name='A', cluster=True) as G:
    G.add_node('s', label='', shape='none')
    G.add_node('e', label='', color='black', shape='circle', penwidth=3)
    for i in range(10):
        G.add_node(i, label='', color='black', shape='circle')
    G.add_edge('s', 0, color='black')
    G.add_edge(0, 1, color='black')
    G.add_edge(1, 2, color='black', label='a')
    G.add_edge(2, 3, color='black')
    G.add_edge(3, 3, color='black', label='b')
    G.add_edge(3, 4, color='black')
    G.add_edge(4, 1, color='black', label='a')
    G.add_edge(1, 5, color='black', label='b')
    G.add_edge(5, 6, color='black', label='a')
    G.add_edge(5, 6, color='black', label='b')
    G.add_edge(6, 7, color='black', label='a')
    G.add_edge(6, 1, color='black', label='b')
    G.add_edge(7, 8, color='black')
    G.add_edge(8, 8, color='black', label='b')
    G.add_edge(8, 9, color='black')
    G.add_edge(9, 1, color='black', label='a')
    G.add_edge(1, 'e', color='black')

    G.draw("test.png","png", prog="neato")
    draw(G, prog='neato', width=600)

In [ ]:
graph = pgv.AGraph(
    strict=False,
    directed=True,
    compound=True,
    rankdir='LR',
    ranksep=1
)
with graph.add_subgraph(name='A', cluster=True) as G:
    G.add_node('s', label='', shape='none')
    G.add_node('e', label=abc[16], color='black', shape='circle', penwidth=3)
    for i in range(16):
        G.add_node(i, label=abc[i], color='black', shape='circle')
    G.add_edge('s', 10, color='black')
    G.add_edge(10, 11, color='black', label='a')
    G.add_edge(10, 0, color='black', label='b')
    G.add_edge(11, 12, color='black')
    G.add_edge(12, 12, color='black', label='b')
    G.add_edge(12, 13, color='black')
    G.add_edge(13, 0, color='black', label='a')
    G.add_edge(0, 1, color='black')
    G.add_edge(1, 2, color='black', label='a')
    G.add_edge(2, 3, color='black')
    G.add_edge(3, 3, color='black', label='b')
    G.add_edge(3, 4, color='black')
    G.add_edge(4, 1, color='black', label='a')
    G.add_edge(1, 5, color='black', label='b')
    G.add_edge(5, 6, color='black', label='a')
    G.add_edge(5, 6, color='black', label='b')
    G.add_edge(6, 7, color='black', label='a')
    G.add_edge(6, 1, color='black', label='b')
    G.add_edge(7, 8, color='black')
    G.add_edge(8, 8, color='black', label='b')
    G.add_edge(8, 9, color='black')
    G.add_edge(9, 1, color='black', label='a')
    G.add_edge(1, 14, color='black')
    G.add_edge(14, 15, color='black', label='b')
    G.add_edge(15, 'e', color='black', label='a')
    G.add_edge(15, 'e', color='black', label='b')
    G.add_edge(14, 'e', color='black', label=empty)
    G.add_edge(10, 'e', color='black', label=empty)

    G.draw("test.png","png", prog="neato")
    draw(G, prog='neato', width=400)

In [ ]:
# task4_result = MooreStateMachine(
#     in_abc='ab'+empty,
#     out_abc='01',
#     state_abc='ABCDEFGHIJKLMNOPQ',
#     l_func={
#         s: int(s=='Q')
#         for s in 'ABCDEFGHIJKLMNOPQ'
#     },
#     d_func={
#         ('K', 'a'): 'L',
#         ('K', 'b'): 'A',
#         ('K', empty): 'Q',
#         ('L', empty): 'M',
#         ('M', 'b'): 'M',
#         ('M', empty): 'N',
#         ('N', 'a'): 'A',
#         ('A', empty): 'B',
#         ('B', empty): 'O',
#         ('O', empty): 'Q',
#         ('O', 'b'): 'P',
#         ('P', 'a'): 'Q',
#         ('P', 'b'): 'Q',
#         ('B', 'a'): 'C',
#         ('C', empty): 'D',
#         ('D', 'b'): 'D',
#         ('D', empty): 'E',
#         ('E', 'a'): 'B',
#         ('B', 'b'): 'F',
#         ('F', 'a'): 'G',
#         ('F', 'b'): 'G',
#         ('G', 'a'): 'H',
#         ('H', empty): 'I',
#         ('I', 'b'): 'I',
#         ('I', empty): 'J',
#         ('J', 'a'): 'B',
#     },
#     start_state='K',
#     empty_input=empty
# )
# # minim = task4_result.get_minimizer()
# print(str(task4_result.get_table()).replace('NaN', '  -'))
# draw(stan.get_graph('1'),  prog='circo', width=500)
# draw(task4_result.get_graph('1'), prog='neato', width=500)
# task4_result.get_graph('1').draw('test.png','png','neato')

In [ ]:
# task4_res_min = task4_result.get_minimizer().get_minimal()
# # task4_ultimate = task4_res_mealy | task4_ori_mealy
# print(task4_res_min.get_table())
# draw(task4_res_min.get_graph('1'), moore.get_graph('1'))
# # draw(task4_res_mealy.get_graph(), task4_ori_mealy.get_graph(), task4_ultimate.get_graph())

### -

In [ ]:
task4_result = Undetermined(
    in_abc='ab',
    state_abc='KMBDFPGI',
    # state_abc='KMBDFPGQI',
    d_func={
        ('K','a'): {'M'},
        ('K','b'): {'B'},
        # ('K','ε'): {'Q'},
        # ('K','ε'): {'Q'},
        ('M','a'): {'B'},
        ('M','b'): {'M'},
        ('B','a'): {'D'},
        ('B','b'): {'F','P'},
        ('F','a'): {'G'},
        ('F','b'): {'G'}, 
        ('P','a'): {'K'},
        ('P','b'): {'K'}, 
        # ('P','a'): {'Q'},
        # ('P','b'): {'Q'}, 
        ('G','a'): {'I'},
        ('I','a'): {'B'},
        ('I','b'): {'I'}, 
        ('D','a'): {'B'},
        ('D','b'): {'D'}, 
    },
    start_states=('K', ),
    fin_states=('K','B',)
)
t4_moore = task4_result.to_moore()
t4_minimizer = t4_moore.get_minimizer()
t4_min = t4_minimizer.get_minimal()
print(t4_min.get_table())
draw(t4_min.get_graph('1'), width=400, prog='neato')

In [ ]:
temp = minim.to_moore()
print(temp.get_table())
draw(temp.get_graph('1'), width=500, prog='circo')

In [ ]:
temp

## 5

In [ ]:
temp = Undetermined(
    in_abc='ab',
    start_states={1, 3},
    state_abc={1, 2, 3, 4},
    fin_states={3, 4},
    d_func={
        (1, 'a'): {2, 3},
        (2, 'a'): {1},
        (2, 'b'): {4},
        (3, 'b'): {2},
        (4, 'a'): {3},
    }
)
pprint(temp.get_g_and_m())
pprint(temp.get_table())
pprint(temp.in_abc)
pprint(temp.state_abc)
pprint(temp.start_states)
pprint(temp.fin_states)
pprint(temp.d_func)
draw(temp,width=400, prog='circo')
temp1 = temp.to_moore()
pprint(temp1.get_table())
draw(temp1.get_graph('1'),width=400, prog='circo')
temp2 = temp1.get_minimizer().get_minimal()
draw(temp2.get_graph('1'),width=400, labels = ['lal'], prog='circo')
